In [ ]:
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import numpy as np
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

In [ ]:
from tabpfn.scripts.distill_mlp import TorchMLP, DistilledTabPFNMLP
from tabpfn.scripts.transformer_prediction_interface import TabPFNClassifier

In [ ]:
TorchMLP(n_epochs=100).fit(X_train, y_train).score(X_test, y_test)

In [ ]:
TorchMLP(n_epochs=100).fit(X_train_s, y_train).score(X_test_s, y_test)

In [ ]:
DistilledTabPFNMLP(n_epochs=100).fit(X_train, y_train).score(X_test, y_test)

In [ ]:
DistilledTabPFNMLP(n_epochs=100).fit(X_train_s, y_train).score(X_test_s, y_test)

In [ ]:
TabPFNClassifier().fit(X_train_s, y_train).score(X_test_s, y_test)

In [ ]:
DistilledTabPFNMLP(n_epochs=100, temperature=10).fit(X_train_s, y_train).score(X_test_s, y_test)

In [ ]:
from tabpfn.datasets import load_openml_list, open_cc_dids, open_cc_valid_dids, test_dids_classification

cc_valid_datasets_multiclass, cc_valid_datasets_multiclass_df = load_openml_list(open_cc_valid_dids, multiclass=True, shuffled=True, filter_for_nan=False, max_samples = 10000, num_feats=100, return_capped=True)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier


def make_mlp(categorical_features):
    cont_pipe = make_pipeline(StandardScaler(), SimpleImputer())
    preprocess = make_column_transformer((OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features), remainder=cont_pipe)
    return make_pipeline(preprocess, TorchMLP(n_epochs=100))

def make_distilled_tabpfn(categorical_features):
    cont_pipe = make_pipeline(StandardScaler(), SimpleImputer())
    preprocess = make_column_transformer((OneHotEncoder(handle_unknown='ignore', sparse_output=False, max_categories=10), categorical_features), remainder=cont_pipe)
    return make_pipeline(preprocess, DistilledTabPFNMLP(n_epochs=100))

def make_distilled_tabpfn_ht(categorical_features):
    cont_pipe = make_pipeline(StandardScaler(), SimpleImputer())
    preprocess = make_column_transformer((OneHotEncoder(handle_unknown='ignore', sparse_output=False, max_categories=10), categorical_features), remainder=cont_pipe)
    return make_pipeline(preprocess, DistilledTabPFNMLP(n_epochs=100, temperature=10))

def make_distilled_tabpfn_temp_4(categorical_features):
    cont_pipe = make_pipeline(StandardScaler(), SimpleImputer())
    preprocess = make_column_transformer((OneHotEncoder(handle_unknown='ignore', sparse_output=False, max_categories=10), categorical_features), remainder=cont_pipe)
    return make_pipeline(preprocess, DistilledTabPFNMLP(n_epochs=100, temperature=4))

In [ ]:
models = {'mlp': make_mlp,
          'distilled_tabpfn': make_distilled_tabpfn,
         'distilled_tabpfn_ht': make_distilled_tabpfn_ht,

          'logreg': make_logreg,
            'knn': make_knn,
            'hgb': make_hgb,
            'rf': make_rf,
            'tabpfn': make_tabpfn}

In [ ]:

from collections import defaultdict
all_scores = defaultdict(dict)
for ds_name, X, y, categorical_features, _, _ in cc_valid_datasets_multiclass:
    print(ds_name)
    for model_name, model_creator in models.items():
        clf = model_creator(categorical_features)
        if X.shape[1] > 100:
            X = X[:, :100]
        try:
            scores = cross_validate(clf, X, y, scoring="roc_auc_ovo")
            score = scores['test_score'].mean()
        except:
            score = np.NaN
        all_scores[ds_name][model_name] = score

In [ ]:
import pandas as pd
results = pd.DataFrame.from_dict(all_scores).T

In [ ]:
results.isna().sum()

In [ ]:
results.to_csv("sklearn_implementation_baselines_auc.cvs")

In [ ]:
results.dropna()

In [ ]:
bla  = results.dropna().stack().reset_index()
bla.columns = ['dataset_name', 'classifier_name', 'accuracy']
bla

In [ ]:
from critical_differences import draw_cd_diagram
res = draw_cd_diagram(bla, title="OVO ROC AUC")

In [ ]:
from collections import defaultdict
all_scores = defaultdict(dict)

from collections import defaultdict
for ds_name, X, y, categorical_features, _, _ in cc_valid_datasets_multiclass:
    print(ds_name)
    for model_name, model_creator in models.items():
        clf = model_creator(categorical_features)
        if X.shape[1] > 100:
            X = X[:, :100]
        try:
            scores = cross_validate(clf, X, y, scoring="roc_auc_ovo", error_score="raise")
            score = scores['test_score'].mean()
        except Exception as e:
            print(e)
            score = np.NaN
        all_scores[ds_name][model_name] = score

In [ ]:
all_scores

In [ ]:
import pandas as pd
results = pd.read_csv("sklearn_implementation_baselines_auc.cvs", index_col=0)

In [ ]:
results_new = pd.DataFrame.from_dict(all_scores).T

In [ ]:
results_new

In [ ]:
results_new.isna().sum()

In [ ]:
results

In [ ]:
res = pd.concat([results, results_new], axis=1)

In [ ]:
res

In [ ]:
res.to_csv("distillation_comparison_lgbm.csv")

In [ ]:
import pandas as pd
res = pd.read_csv("distillation_comparison.csv", index_col=0)

In [ ]:
res.isna().sum()

In [ ]:
grr = res.isna().sum(axis=1)
grr[grr>0]

In [ ]:
res.dropna()

In [ ]:
bla  = res.drop(columns="hgb").dropna().stack().reset_index()
bla.columns = ['dataset_name', 'classifier_name', 'accuracy']
bla

In [ ]:
from critical_differences import draw_cd_diagram
draw_cd_diagram(bla, title="OVO ROC AUC")

In [ ]:
models

In [ ]:
all_scores

In [ ]:
np.isnan(np.nan)

In [ ]:
from tabpfn.scripts.evaluate_baselines_sklearn import make_logreg, make_knn, make_hgb, make_rf, make_tabpfn, make_mlp, make_distilled_tabpfn

In [ ]:
%pip install lightgbm

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
def make_lgbm(categorical_features):
    return LGBMClassifier(categorical_features=categorical_features)

In [ ]:
models = {'mlp': make_mlp,
          'distilled_tabpfn': make_distilled_tabpfn,
         'distilled_tabpfn_ht': make_distilled_tabpfn_ht,
         'distilled_tabpfn_temp4': make_distilled_tabpfn_temp_4,

          'logreg': make_logreg,
            'knn': make_knn,
            'hgb': make_hgb,
            'rf': make_rf,
            'tabpfn': make_tabpfn}

In [ ]:
res['distilled_tabpfn_temp4'] = np.NaN

In [ ]:
res

In [ ]:
from sklearn.model_selection import cross_validate

for ds_name, X, y, categorical_features, _, _ in cc_valid_datasets_multiclass:
    print(ds_name)
    for model_name, model_creator in models.items():
        if not np.isnan(res.loc[ds_name, model_name]):
            continue
        clf = model_creator(categorical_features)
        if X.shape[1] > 100:
            X = X[:, :100]
        try:
            scores = cross_validate(clf, X, y, scoring="roc_auc_ovo", error_score="raise")
            score = scores['test_score'].mean()
        except Exception as e:
            print(str(e))
            score = np.NaN
        res.loc[ds_name, model_name] = score

In [ ]:
res.isna().sum()